# Lunar Survival Challenge Multi-Agent System Implementation
Implemented a comprehensive collaborative agent-based system for the lunar survival challenge. The system follows our redesigned workflow with paired agent teams and a focus on deeper collaboration.

In [16]:
!pip3 install fpdf

  Using cached fpdf-1.7.2.tar.gz (39 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [1 lines of output]
  ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [13]:
import os
import json
import logging
from typing import List, Dict, Any, Optional, Tuple
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from fpdf2 import FPDF  # For PDF generation

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.FileHandler("lunar_survival_agents.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

class Agent:
    """Base agent class for lunar survival challenge."""
    
    def __init__(self, 
                 instruction: str, 
                 role: str, 
                 expertise_description: str,
                 examples: Optional[List[Dict[str, str]]] = None):
        """
        Initialize an LLM-based agent with a specific role.
        
        Args:
            instruction: The system instruction for the agent
            role: The role of the agent (e.g., "Science Analyst")
            expertise_description: Description of the agent's expertise
            examples: Optional examples to include in the prompt
        """
        self.instruction = instruction
        self.role = role
        self.expertise_description = expertise_description
        self.examples = examples or []
        self.conversation_history = []
        
        # Initialize LLM
        self.client = AzureChatOpenAI(
            azure_deployment="VARELab-GPT4o",
            api_key=os.environ.get('AZURE_OPENAI_VARE_KEY'),
            api_version="2024-08-01-preview",
            azure_endpoint=os.environ.get('AZURE_ENDPOINT'),
            temperature=0.5
        )
        
        # Build initial system message
        self.messages = [
            {"role": "system", "content": self._build_system_prompt()}
        ]
        
        # Add example conversations if provided
        if self.examples:
            for example in self.examples:
                self.messages.append({"role": "user", "content": example['question']})
                self.messages.append({
                    "role": "assistant", 
                    "content": example['answer'] + "\n\n" + example.get('reason', '')
                })
                
        logger.info(f"Initialized {self.role} agent")
    
    def _build_system_prompt(self) -> str:
        """Build the system prompt for the agent."""
        prompt = f"You are a {self.role} who {self.expertise_description}. {self.instruction}"
        return prompt
    
    def chat(self, message: str) -> str:
        """
        Send a message to the agent and get a response.
        
        Args:
            message: The message to send to the agent
            
        Returns:
            The agent's response
        """
        logger.info(f"[{self.role}] Received message: {message[:100]}...")
        
        # Add the user message to the conversation
        self.messages.append({"role": "user", "content": message})
        
        # Get response from LLM
        response = self.client.predict_messages(
            messages=self.messages
        )
        
        # Extract and store the response
        assistant_message = response.content
        self.messages.append({"role": "assistant", "content": assistant_message})
        self.conversation_history.append({"user": message, "assistant": assistant_message})
        
        logger.info(f"[{self.role}] Responded: {assistant_message[:100]}...")
        
        return assistant_message
    
    def get_conversation_history(self) -> List[Dict[str, str]]:
        """Get the conversation history."""
        return self.conversation_history


class WebResearchAgent(Agent):
    """Agent specialized in web research for gathering information."""
    
    def __init__(self):
        super().__init__(
            instruction="You specialize in gathering factual information from the web. Focus on obtaining accurate data about the lunar environment and survival in space.",
            role="Web Researcher",
            expertise_description="specializes in gathering and synthesizing information from web sources"
        )
    
    def research(self, query: str) -> str:
        """
        Perform research on a specific query.
        
        Args:
            query: The research query
            
        Returns:
            Research findings
        """
        # In a real implementation, this would use langchain's WebBrowser tool
        # Since we can't directly use that here, we'll simulate with a focused prompt
        
        research_prompt = f"""
        You need to research: "{query}"
        
        As a Web Researcher, your task is to provide accurate information as if you've researched this topic.
        
        Focus on:
        1. Scientific facts about the lunar environment
        2. NASA procedures and protocols
        3. Survival requirements in space
        4. Properties of survival equipment in space
        
        Provide a comprehensive synthesis of this information in a factual, well-structured format.
        Include specific details about the lunar environment such as:
        - No atmosphere (vacuum)
        - Extreme temperature variations
        - Radiation exposure
        - Low gravity (1/6 of Earth)
        - Lunar dust properties
        
        Cite relevant NASA knowledge about survival priorities in space.
        """
        
        logger.info(f"[Web Researcher] Researching: {query}")
        research_results = self.chat(research_prompt)
        
        return research_results


class Team:
    """Represents a collaborative team of agents working together."""
    
    def __init__(self, 
                 name: str, 
                 goal: str, 
                 members: List[Dict[str, str]],
                 examples: Optional[List[Dict[str, str]]] = None):
        """
        Initialize a team of agents.
        
        Args:
            name: Name of the team
            goal: The shared goal of the team
            members: List of member information (role, expertise_description)
            examples: Optional examples to include in agent prompts
        """
        self.name = name
        self.goal = goal
        self.examples = examples
        self.agents = []
        self.team_conversation = []
        
        logger.info(f"Initializing {name} team with goal: {goal}")
        
        # Create agents based on member information
        for member_info in members:
            agent = Agent(
                instruction=f"You are part of the {name} team. Your goal is to {goal}.",
                role=member_info['role'],
                expertise_description=member_info['expertise_description'],
                examples=examples
            )
            self.agents.append(agent)
    
    def collaborate(self, topic: str, rounds: int = 3) -> List[Dict[str, str]]:
        """
        Facilitate collaboration between team members.
        
        Args:
            topic: The topic for collaboration
            rounds: Number of discussion rounds
            
        Returns:
            Conversation records
        """
        logger.info(f"[{self.name} Team] Starting collaboration on: {topic}")
        
        conversation_records = []
        
        # Initial prompt to each agent
        initial_responses = []
        for agent in self.agents:
            response = agent.chat(
                f"As a {agent.role}, please provide your initial thoughts on: {topic}"
            )
            initial_responses.append({
                "role": agent.role,
                "response": response
            })
            conversation_records.append({
                "round": 0,
                "agent": agent.role,
                "message": response
            })
        
        # Collaboration rounds
        for round_num in range(1, rounds + 1):
            logger.info(f"[{self.name} Team] Starting collaboration round {round_num}")
            
            # Each agent responds to the previous round
            for i, agent in enumerate(self.agents):
                # Get the other agent's response from previous round
                other_agent = self.agents[1 - i]  # For 2 agents
                other_response = initial_responses[1 - i]["response"] if round_num == 1 else conversation_records[-2]["message"]
                
                prompt = f"""
                Your teammate ({other_agent.role}) said: 
                
                "{other_response}"
                
                Based on your expertise as a {agent.role} and considering your teammate's perspective, 
                please refine your analysis or provide additional insights on {topic}.
                If you agree with certain points, acknowledge them.
                If you have different perspectives, explain your reasoning.
                Work toward developing an integrated viewpoint with your teammate.
                """
                
                response = agent.chat(prompt)
                conversation_records.append({
                    "round": round_num,
                    "agent": agent.role,
                    "message": response
                })
        
        # Final consensus
        logger.info(f"[{self.name} Team] Generating final consensus")
        
        consensus_prompt = f"""
        Review the entire discussion your team has had about {topic}.
        As a {self.agents[0].role}, collaborate with your teammate to provide a final consensus view that
        integrates both of your perspectives. This should represent your team's collective expertise and judgment.
        """
        
        consensus_response = self.agents[0].chat(consensus_prompt)
        conversation_records.append({
            "round": "consensus",
            "agent": f"{self.name} Team",
            "message": consensus_response
        })
        
        self.team_conversation = conversation_records
        
        # Log full conversation to file
        self._log_conversation_to_file()
        
        return conversation_records
    
    def _log_conversation_to_file(self):
        """Log the full conversation to a separate file for review."""
        filename = f"{self.name.replace(' ', '_').lower()}_conversation.txt"
        
        with open(filename, "w", encoding="utf-8") as f:
            f.write(f"=== {self.name} Team Conversation ===\n\n")
            f.write(f"Goal: {self.goal}\n\n")
            
            for entry in self.team_conversation:
                f.write(f"Round: {entry['round']}\n")
                f.write(f"Agent: {entry['agent']}\n")
                f.write(f"Message:\n{entry['message']}\n\n")
                f.write("-" * 80 + "\n\n")
        
        logger.info(f"[{self.name} Team] Conversation log saved to {filename}")


ModuleNotFoundError: No module named 'fpdf2'

## LunarSurvivalSystem class

In [19]:
class LunarSurvivalSystem:
    """Main system for lunar survival challenge."""
    
    def __init__(self):
        """Initialize the lunar survival system with teams and meta agent."""
        
        logger.info("Initializing Lunar Survival System")
        
        # Define correct lunar survival items
        self.survival_items = [
            {"name": "Box of matches", "description": "Standard matches for lighting fire"},
            {"name": "Food concentrate", "description": "Dehydrated food for consumption"},
            {"name": "50 feet of nylon rope", "description": "Strong utility rope"},
            {"name": "Parachute silk", "description": "Large sheet of parachute silk"},
            {"name": "Portable heating unit", "description": "Device for generating heat"},
            {"name": "Two .45 caliber pistols", "description": "Firearms with ammunition"},
            {"name": "One case of dehydrated milk", "description": "Powdered milk for consumption"},
            {"name": "Two 100 lb. tanks of oxygen", "description": "Compressed oxygen supply"},
            {"name": "Stellar map", "description": "Map of the moon's constellation"},
            {"name": "Self-inflating life raft", "description": "Inflatable raft"},
            {"name": "Magnetic compass", "description": "Navigation tool"},
            {"name": "20 liters of water", "description": "Potable water supply"},
            {"name": "Signal flares", "description": "Visual emergency signaling devices"},
            {"name": "First aid kit, including injection needle", "description": "Medical supplies"},
            {"name": "Solar-powered FM receiver-transmitter", "description": "Communication device"}
        ]
        
        # Create web research agent
        self.web_researcher = WebResearchAgent()
        
        # Create knowledge acquisition team
        self.knowledge_team = Team(
            name="Knowledge Acquisition",
            goal="gather and analyze information about the lunar environment and human survival needs",
            members=[
                {
                    "role": "Science Analyst",
                    "expertise_description": "specializes in lunar physics, environmental conditions, and how items function in the lunar environment"
                },
                {
                    "role": "Resource Analyst",
                    "expertise_description": "specializes in human physiological needs, survival requirements, and resource management in extreme conditions"
                }
            ]
        )
        
        # Create technical reasoning team
        self.technical_team = Team(
            name="Technical Reasoning",
            goal="evaluate items based on scientific principles and established space procedures",
            members=[
                {
                    "role": "Science Reasoner",
                    "expertise_description": "applies scientific knowledge to evaluate how effectively items will function in the lunar environment"
                },
                {
                    "role": "Protocol Reasoner",
                    "expertise_description": "applies standard space agency procedures and protocols to evaluate items based on established guidelines"
                }
            ]
        )
        
        # Create practical reasoning team
        self.practical_team = Team(
            name="Practical Reasoning",
            goal="evaluate items based on survival needs and contingency planning",
            members=[
                {
                    "role": "Resource Reasoner",
                    "expertise_description": "optimizes resource allocation based on survival hierarchy and human needs"
                },
                {
                    "role": "Creative Reasoner",
                    "expertise_description": "considers edge cases, alternative item uses, and unexpected scenarios"
                }
            ]
        )
        
        # Create meta reasoning agent
        self.meta_agent = Agent(
            instruction="You are responsible for integrating multiple team perspectives and making final decisions.",
            role="Meta Reasoner",
            expertise_description="specializes in holistic reasoning, conflict resolution, and integrated decision making"
        )
        
        # Track system state
        self.knowledge_base = {}
        self.technical_ranking = []
        self.practical_ranking = []
        self.final_ranking = []
        self.justifications = {}
        
    #################################################################################

    def _web_research_phase(self):
        """
        Execute the web research phase to gather information about lunar survival.
        """
        logger.info("Starting Web Research Phase...")

        # Research lunar environment
        lunar_env_research = self.web_researcher.research(
            "Lunar environment characteristics and challenges for human survival on the moon"
        )

        # Research survival principles
        survival_research = self.web_researcher.research(
            "NASA protocols and guidelines for survival in space emergencies"
        )

        # Research specific items
        items_research = self.web_researcher.research(
            "Effectiveness of various equipment in lunar environment: oxygen tanks, water, signal flares, communication devices, matches, food, compass, weapons"
        )

        # Store research results
        self.knowledge_base["web_research"] = {
            "lunar_environment": lunar_env_research,
            "survival_protocols": survival_research,
            "items_analysis": items_research
        }

        # Save research to file for review
        with open("web_research_results.txt", "w", encoding="utf-8") as f:
            f.write("=== Web Research Results ===\n\n")
            f.write("LUNAR ENVIRONMENT RESEARCH:\n")
            f.write(lunar_env_research)
            f.write("\n\n" + "-" * 80 + "\n\n")
            f.write("SURVIVAL PROTOCOLS RESEARCH:\n")
            f.write(survival_research)
            f.write("\n\n" + "-" * 80 + "\n\n")
            f.write("ITEMS ANALYSIS RESEARCH:\n")
            f.write(items_research)

        logger.info("Web Research Phase completed.")
        return self.knowledge_base["web_research"]

    ##################################################################################

    def _knowledge_acquisition_phase(self):
        """
        Execute the knowledge acquisition phase where agents gather and integrate 
        knowledge about lunar environment and survival needs.
        """
        logger.info("Starting Knowledge Acquisition Phase...")

        # First, get web research
        web_research = self._web_research_phase()

        # Step 1: Gather lunar environmental information with web research input
        lunar_environment_prompt = f"""
        As a Science Analyst, your task is to gather comprehensive information about the lunar environment.

        Use this web research as a starting point:
        {web_research['lunar_environment']}

        Focus on:
        1. Physical conditions (vacuum, temperature extremes, radiation, gravity)
        2. Surface characteristics (terrain, dust, etc.)
        3. Day/night cycle and their implications
        4. How these conditions would affect the functioning of various items and human survival

        Review the following items that need to be evaluated:
        {', '.join([item["name"] for item in self.survival_items])}

        Provide a structured knowledge base that will be useful for evaluating these specific survival items.
        """

        # Step 2: Gather human survival needs information with web research input
        survival_needs_prompt = f"""
        As a Resource Analyst, your task is to gather comprehensive information about human survival needs in the lunar environment.

        Use this web research as a starting point:
        {web_research['survival_protocols']}

        Focus on:
        1. Basic physiological needs (oxygen, water, food, temperature regulation)
        2. Timeframes for various survival needs (how long before critical failure)
        3. Special considerations for the lunar environment
        4. Hierarchy of survival priorities according to NASA protocols

        Review the following items that need to be evaluated:
        {', '.join([item["name"] for item in self.survival_items])}

        Provide a structured knowledge base that will be useful for evaluating these specific survival items.
        """

        # Get responses from knowledge team
        env_response = self.knowledge_team.agents[0].chat(lunar_environment_prompt)
        needs_response = self.knowledge_team.agents[1].chat(survival_needs_prompt)

        # Step 3: Collaborative knowledge integration
        integration_prompt = f"""
        You need to collaborate to create an integrated knowledge base for lunar survival.

        The Science Analyst has provided information about the lunar environment:
        {env_response}

        The Resource Analyst has provided information about human survival needs:
        {needs_response}

        Additional items analysis from research:
        {web_research['items_analysis']}

        Work together to create an integrated knowledge framework that connects:
        1. Environmental challenges with human survival needs
        2. How different environmental factors affect survival priorities
        3. Critical timeframes and dependencies
        4. Specific analysis of how each item on our list would function on the moon

        This integrated knowledge will be used to evaluate survival items, so make it practical and comprehensive.

        The items to be evaluated are:
        {', '.join([item["name"] for item in self.survival_items])}
        """

        # Collaborative knowledge integration through team discussion
        knowledge_integration = self.knowledge_team.collaborate(
            topic=integration_prompt,
            rounds=2
        )

        # Extract the final consensus as our knowledge base
        integrated_knowledge = knowledge_integration[-1]["message"]

        # Store structured knowledge in our system
        self.knowledge_base = {
            "web_research": web_research,
            "lunar_environment": env_response,
            "survival_needs": needs_response,
            "integrated_knowledge": integrated_knowledge
        }

        # Format knowledge for reasoning teams
        self._prepare_knowledge_for_reasoning_teams()

        logger.info("Knowledge Acquisition Phase completed.")
        return self.knowledge_base

    ##################################################################################

    def _prepare_knowledge_for_reasoning_teams(self):
        """
        Prepare knowledge in a format suitable for reasoning teams.
        This includes extracting key principles and facts from the knowledge base.
        """
        # Extract key environmental principles
        env_principles_prompt = f"""
        Based on this knowledge about the lunar environment:

        {self.knowledge_base['lunar_environment']}

        Please extract 10-15 key scientific principles that are most relevant for evaluating these survival items:
        {', '.join([item["name"] for item in self.survival_items])}

        Format as a numbered list of concise statements that can be directly used by the reasoning teams.
        """
        env_principles = self.knowledge_team.agents[0].chat(env_principles_prompt)

        # Extract key survival principles
        survival_principles_prompt = f"""
        Based on this knowledge about human survival needs:

        {self.knowledge_base['survival_needs']}

        Please extract 10-15 key survival principles that are most relevant for evaluating these survival items:
        {', '.join([item["name"] for item in self.survival_items])}

        Format as a numbered list of concise statements that can be directly used by the reasoning teams.
        Include time-critical factors and priority hierarchies.
        """
        survival_principles = self.knowledge_team.agents[1].chat(survival_principles_prompt)

        # Add to knowledge base
        self.knowledge_base["environmental_principles"] = env_principles
        self.knowledge_base["survival_principles"] = survival_principles

        # Create item-specific knowledge
        item_analysis_prompt = f"""
        Based on the lunar environment information and survival principles:

        Environmental principles:
        {env_principles}

        Survival principles:
        {survival_principles}

        For each of these survival items, provide a brief analysis of:
        1. How it would function in the lunar environment
        2. Its potential survival utility
        3. Any special considerations

        Be concise but thorough. This information will be used by reasoning teams to evaluate the items.

        Items to analyze:
        {', '.join([item["name"] for item in self.survival_items])}
        """

        # Collaborative item analysis
        item_analysis_discussion = self.knowledge_team.collaborate(
            topic=item_analysis_prompt,
            rounds=1
        )

        # Extract and store the item analysis
        self.knowledge_base["item_analysis"] = item_analysis_discussion[-1]["message"]

        # Save formatted knowledge to file for review
        with open("knowledge_base_for_reasoning.txt", "w", encoding="utf-8") as f:
            f.write("=== Knowledge Base for Reasoning Teams ===\n\n")
            f.write("ENVIRONMENTAL PRINCIPLES:\n")
            f.write(env_principles)
            f.write("\n\n" + "-" * 80 + "\n\n")
            f.write("SURVIVAL PRINCIPLES:\n")
            f.write(survival_principles)
            f.write("\n\n" + "-" * 80 + "\n\n")
            f.write("ITEM-SPECIFIC ANALYSIS:\n")
            f.write(self.knowledge_base["item_analysis"])

    
    
    ##################################################################################
    
    def run(self) -> Dict[str, Any]:
        """
        Run the complete lunar survival challenge analysis.
        
        Returns:
            Dict containing final rankings and justifications
        """
        # Step 1: Knowledge Acquisition Phase
        self._knowledge_acquisition_phase()
        
        # Step 2: Technical Team Reasoning
        self._technical_reasoning_phase()
        
        # Step 3: Practical Team Reasoning
        self._practical_reasoning_phase()
        
        # Step 4: Cross-Team Integration
        self._cross_team_integration()
        
        # Step 5: Meta Reasoning and Final Ranking
        self._meta_reasoning_phase()
        
        # Prepare and return results
        results = {
            "final_ranking": self.final_ranking,
            "justifications": self.justifications,
            "knowledge_base": self.knowledge_base,
            "technical_ranking": self.technical_ranking,
            "practical_ranking": self.practical_ranking
        }
        
        return results
    
    
    #################################################################################

    def _technical_reasoning_phase(self):
        """
        Execute the technical reasoning phase where the Science Reasoner and Protocol Reasoner
        collaborate to evaluate items based on scientific principles and space procedures.
        """
        logger.info("Starting Technical Reasoning Phase...")

        # Prepare item list for evaluation
        item_descriptions = "\n".join([f"- {item['name']}: {item['description']}" for item in self.survival_items])

        # Create a collaborative reasoning prompt using knowledge from acquisition phase
        reasoning_prompt = f"""
        You are tasked with evaluating lunar survival items from a technical perspective.

        Environmental Knowledge:
        {self.knowledge_base['environmental_principles']}

        Item Analysis:
        {self.knowledge_base['item_analysis']}

        Items to evaluate:
        {item_descriptions}

        As a collaborative team:
        1. The Science Reasoner should focus on how each item functions in the lunar environment
        2. The Protocol Reasoner should focus on how each item aligns with standard space procedures
        3. Together, develop an integrated technical evaluation

        Your task is to collectively rank these items from most important (1) to least important (15) 
        for lunar survival based on your technical expertise.

        Consider this scenario: A spacecraft has crash-landed on the moon, approximately 200 miles from the rendezvous point where another spacecraft is waiting. The crew needs to travel to the rendezvous point with only these items available.

        For each item, provide:
        1. Its rank (from 1 to 15, with 1 being most important)
        2. A brief technical justification focusing on scientific principles and protocols
        3. A confidence score (1-10)

        Work together to produce an integrated technical ranking.
        """

        # Facilitate collaborative reasoning in the technical team
        technical_collaboration = self.technical_team.collaborate(
            topic=reasoning_prompt,
            rounds=3  # More rounds for thorough discussion
        )

        # Extract the consensus ranking
        technical_consensus = technical_collaboration[-1]["message"]

        # Save the complete discussion to file for review
        with open("technical_reasoning_full_discussion.txt", "w", encoding="utf-8") as f:
            f.write("=== Technical Reasoning Team - Full Discussion ===\n\n")
            for entry in technical_collaboration:
                f.write(f"Round: {entry['round']}\n")
                f.write(f"Agent: {entry['agent']}\n")
                f.write(f"Message:\n{entry['message']}\n\n")
                f.write("-" * 80 + "\n\n")

        # Parse the ranking from the consensus message
        # This is a simplified parsing approach - in a production system, you'd want more robust parsing
        self.technical_ranking = self._parse_ranking(technical_consensus)

        # Save the parsed ranking for review
        with open("technical_ranking_parsed.txt", "w", encoding="utf-8") as f:
            f.write("=== Technical Team - Parsed Rankings ===\n\n")
            for item in self.technical_ranking:
                f.write(f"Rank {item['rank']}: {item['name']}\n")
                f.write(f"Justification: {item['justification']}\n")
                f.write(f"Confidence: {item['confidence']}\n\n")

        logger.info("Technical Reasoning Phase completed.")
        return self.technical_ranking
    
    
    #################################################################################

    def _practical_reasoning_phase(self):
        """
        Execute the practical reasoning phase where the Resource Reasoner and Creative Reasoner
        collaborate to evaluate items based on survival needs and contingency planning.
        """
        logger.info("Starting Practical Reasoning Phase...")

        # Prepare item list for evaluation
        item_descriptions = "\n".join([f"- {item['name']}: {item['description']}" for item in self.survival_items])

        # Create a collaborative reasoning prompt using knowledge from acquisition phase
        reasoning_prompt = f"""
        You are tasked with evaluating lunar survival items from a practical perspective.

        Survival Knowledge:
        {self.knowledge_base['survival_principles']}

        Item Analysis:
        {self.knowledge_base['item_analysis']}

        Items to evaluate:
        {item_descriptions}

        As a collaborative team:
        1. The Resource Reasoner should focus on how each item addresses critical survival needs
        2. The Creative Reasoner should focus on alternative uses and contingency scenarios
        3. Together, develop an integrated practical evaluation

        Consider this scenario: A spacecraft has crash-landed on the moon, approximately 200 miles from the rendezvous point where another spacecraft is waiting. The crew needs to travel to the rendezvous point with only these items available.

        Your task is to collectively rank these items from most important (1) to least important (15) 
        for lunar survival based on your practical expertise.

        For each item, provide:
        1. Its rank (from 1 to 15, with 1 being most important)
        2. A brief practical justification focusing on survival utility and adaptability
        3. A confidence score (1-10)

        Work together to produce an integrated practical ranking.
        """

        # Facilitate collaborative reasoning in the practical team
        practical_collaboration = self.practical_team.collaborate(
            topic=reasoning_prompt,
            rounds=3  # More rounds for thorough discussion
        )

        # Extract the consensus ranking
        practical_consensus = practical_collaboration[-1]["message"]

        # Save the complete discussion to file for review
        with open("practical_reasoning_full_discussion.txt", "w", encoding="utf-8") as f:
            f.write("=== Practical Reasoning Team - Full Discussion ===\n\n")
            for entry in practical_collaboration:
                f.write(f"Round: {entry['round']}\n")
                f.write(f"Agent: {entry['agent']}\n")
                f.write(f"Message:\n{entry['message']}\n\n")
                f.write("-" * 80 + "\n\n")

        # Parse the ranking from the consensus message
        self.practical_ranking = self._parse_ranking(practical_consensus)

        # Save the parsed ranking for review
        with open("practical_ranking_parsed.txt", "w", encoding="utf-8") as f:
            f.write("=== Practical Team - Parsed Rankings ===\n\n")
            for item in self.practical_ranking:
                f.write(f"Rank {item['rank']}: {item['name']}\n")
                f.write(f"Justification: {item['justification']}\n")
                f.write(f"Confidence: {item['confidence']}\n\n")

        logger.info("Practical Reasoning Phase completed.")
        return self.practical_ranking
    
    
    #################################################################################

    def _parse_ranking(self, consensus_text):
        """
        Parse ranking information from consensus text.

        Args:
            consensus_text: Text containing ranking information

        Returns:
            List of dictionaries with ranking information
        """
        logger.info("Parsing ranking from consensus text")

        # Initialize list to store parsed rankings
        parsed_rankings = []

        # Extract all item names for matching
        item_names = [item["name"] for item in self.survival_items]

        # Try to extract items with rankings
        lines = consensus_text.split("\n")
        current_item = {}

        for line in lines:
            line = line.strip()

            # Skip empty lines
            if not line:
                continue

            # Look for rank pattern (number followed by period or bracket)
            if line and (line[0].isdigit() and (len(line) > 1 and (line[1] == '.' or line[1] == ')'))):
                # If we were processing an item, save it before starting a new one
                if current_item and 'name' in current_item:
                    parsed_rankings.append(current_item)
                    current_item = {}

                # Start a new item
                parts = line.split(" ", 1)
                if len(parts) > 1:
                    rank_str = parts[0].rstrip('.):')
                    try:
                        rank = int(rank_str)
                        # Try to match with an item name
                        remainder = parts[1].strip()
                        matched_item = None

                        # Check if any item name is in the remainder
                        for item_name in item_names:
                            if item_name.lower() in remainder.lower():
                                matched_item = item_name
                                break

                        # If no exact match, try to find closest match
                        if not matched_item:
                            for item_name in item_names:
                                # Check for partial matches (e.g., "oxygen tanks" might be written as "oxygen")
                                words = item_name.lower().split()
                                for word in words:
                                    if len(word) > 3 and word in remainder.lower():
                                        matched_item = item_name
                                        break
                                if matched_item:
                                    break

                        # If we found a match, create the item entry
                        if matched_item:
                            current_item = {
                                "rank": rank,
                                "name": matched_item,
                                "justification": remainder,
                                "confidence": 5  # Default confidence
                            }
                        else:
                            # No match found, use the text directly
                            item_text = remainder.split(":", 1)[0].strip()
                            current_item = {
                                "rank": rank,
                                "name": item_text,
                                "justification": remainder[len(item_text):].strip(),
                                "confidence": 5  # Default confidence
                            }
                    except ValueError:
                        # Not a valid rank, skip
                        continue

            # Add to justification if we're processing an item
            elif current_item and 'name' in current_item:
                # Look for confidence pattern
                if "confidence:" in line.lower() or "confidence score:" in line.lower():
                    try:
                        # Extract confidence value
                        conf_parts = line.lower().split("confidence")
                        if len(conf_parts) > 1:
                            conf_text = conf_parts[1].strip()
                            if conf_text.startswith(":"):
                                conf_text = conf_text[1:].strip()
                            # Extract the first number found
                            import re
                            numbers = re.findall(r'\d+(?:\.\d+)?', conf_text)
                            if numbers:
                                conf_value = float(numbers[0])
                                # Normalize to 1-10 scale if needed
                                if conf_value > 0 and conf_value <= 1:
                                    conf_value *= 10
                                current_item["confidence"] = min(10, max(1, conf_value))
                    except:
                        logger.warning(f"Failed to parse confidence from: {line}")
                else:
                    current_item["justification"] += " " + line

        # Add the last item if exists
        if current_item and 'name' in current_item:
            parsed_rankings.append(current_item)

        # Handle missing items - add them with lowest ranks
        found_items = [item["name"] for item in parsed_rankings]
        for item_name in item_names:
            if item_name not in found_items:
                logger.warning(f"Item not found in ranking: {item_name}")
                # Add with a default low rank
                max_rank = max([item["rank"] for item in parsed_rankings]) if parsed_rankings else 0
                parsed_rankings.append({
                    "rank": max_rank + 1,
                    "name": item_name,
                    "justification": "No explicit ranking provided",
                    "confidence": 1
                })

        # Sort by rank
        parsed_rankings.sort(key=lambda x: x["rank"])

        return parsed_rankings

    
    #################################################################################
    
    def _cross_team_integration(self):
        """
        Execute the cross-team integration phase where both teams reconcile their rankings
        and identify areas of agreement and disagreement.
        """
        logger.info("Starting Cross-Team Integration Phase...")

        # Compare rankings to identify agreements and disagreements
        agreement_items, disagreement_items = self._identify_agreements_disagreements()

        logger.info(f"Found {len(agreement_items)} items with agreement and {len(disagreement_items)} items with disagreement")

        # Format the findings for both teams
        technical_ranking_formatted = self._format_ranking(self.technical_ranking)
        practical_ranking_formatted = self._format_ranking(self.practical_ranking)

        disagreements_formatted = ""
        for item in disagreement_items:
            tech_item = next((i for i in self.technical_ranking if i["name"] == item), None)
            prac_item = next((i for i in self.practical_ranking if i["name"] == item), None)

            if tech_item and prac_item:
                rank_diff = abs(tech_item["rank"] - prac_item["rank"])
                disagreements_formatted += f"- {item}: Technical rank {tech_item['rank']}, Practical rank {prac_item['rank']} (Difference: {rank_diff})\n"

        # Create integration prompt
        integration_prompt = f"""
        Both the Technical and Practical teams have evaluated the lunar survival items.

        Technical Team Ranking:
        {technical_ranking_formatted}

        Practical Team Ranking:
        {practical_ranking_formatted}

        There is significant disagreement on these items:
        {disagreements_formatted}

        For each item of disagreement, please:
        1. Have both teams explain their reasoning
        2. Discuss the core factors behind the different evaluations
        3. Try to find common ground or explicit reasons for maintaining different views

        The goal is not to force agreement, but to understand each other's perspectives 
        and provide clear reasoning for the Meta Reasoner to consider.

        Focus especially on items with rank differences of 3 or more positions.
        """

        # Create a temporary team for cross-team integration
        # This includes one agent from each team to represent their team's perspective
        integration_team = Team(
            name="Integration",
            goal="reconcile technical and practical perspectives on lunar survival items",
            members=[
                {
                    "role": "Technical Representative",
                    "expertise_description": "represents the technical team's perspective on item evaluation"
                },
                {
                    "role": "Practical Representative",
                    "expertise_description": "represents the practical team's perspective on item evaluation"
                }
            ]
        )

        # Facilitate cross-team discussion
        integration_discussion = integration_team.collaborate(
            topic=integration_prompt,
            rounds=3  # More rounds for thorough discussion of disagreements
        )

        # Store the integration discussion
        self.integration_discussion = integration_discussion

        # Save discussion to file
        with open("cross_team_integration_discussion.txt", "w", encoding="utf-8") as f:
            f.write("=== Cross-Team Integration Discussion ===\n\n")
            f.write("AGREEMENTS AND DISAGREEMENTS:\n")
            f.write(f"Items with agreement: {', '.join(agreement_items)}\n\n")
            f.write(f"Items with disagreement:\n{disagreements_formatted}\n\n")
            f.write("-" * 80 + "\n\n")
            f.write("FULL DISCUSSION:\n\n")
            for entry in integration_discussion:
                f.write(f"Round: {entry['round']}\n")
                f.write(f"Agent: {entry['agent']}\n")
                f.write(f"Message:\n{entry['message']}\n\n")
                f.write("-" * 80 + "\n\n")

        logger.info("Cross-Team Integration Phase completed.")
        return self.integration_discussion
    
    
    #################################################################################

    def _identify_agreements_disagreements(self):
        """
        Identify items with agreement and disagreement between technical and practical rankings.

        Returns:
            Tuple of (agreement_items, disagreement_items)
        """
        agreement_items = []
        disagreement_items = []

        # Create dictionaries for easier comparison
        tech_dict = {item["name"]: item for item in self.technical_ranking}
        prac_dict = {item["name"]: item for item in self.practical_ranking}

        # Check all items
        for item in self.survival_items:
            name = item["name"]

            # Check if item is in both rankings
            if name in tech_dict and name in prac_dict:
                # Check if ranks are similar (within 2 positions)
                if abs(tech_dict[name]["rank"] - prac_dict[name]["rank"]) <= 2:
                    agreement_items.append(name)
                else:
                    disagreement_items.append(name)
            else:
                # If item is missing from either ranking, consider it a disagreement
                disagreement_items.append(name)

        return agreement_items, disagreement_items

    def _format_ranking(self, ranking):
        """Format ranking for display in prompts."""
        formatted = ""
        for item in sorted(ranking, key=lambda x: x["rank"]):
            formatted += f"{item['rank']}. {item['name']}: {item['justification'].strip()} (Confidence: {item['confidence']})\n\n"
        return formatted
    
    
    #################################################################################

    def _meta_reasoning_phase(self):
        """
        Execute the meta reasoning phase where the Meta Reasoner integrates perspectives
        and makes final decisions.
        """
        logger.info("Starting Meta Reasoning Phase...")

        # Extract the final integration discussion
        integration_consensus = self.integration_discussion[-1]["message"]

        # Format rankings and discussions for meta reasoning
        technical_ranking_formatted = self._format_ranking(self.technical_ranking)
        practical_ranking_formatted = self._format_ranking(self.practical_ranking)

        # Create meta reasoning prompt
        meta_prompt = f"""
        As the Meta Reasoner, your task is to integrate the technical and practical perspectives
        on lunar survival items and make final decisions.

        Here is a summary of key knowledge:
        {self.knowledge_base['integrated_knowledge'][:1000]}... (abbreviated)

        Technical Team Ranking:
        {technical_ranking_formatted}

        Practical Team Ranking:
        {practical_ranking_formatted}

        Integration Discussion Summary:
        {integration_consensus}

        Your task:
        1. Create a final ranking of all 15 items from most important (1) to least important (15)
        2. For each item, provide a comprehensive justification that integrates both technical and practical perspectives
        3. Address any significant disagreements and explain your reasoning for the final position
        4. Assign a final confidence score (1-10) for each item

        The items to rank are:
        {', '.join([item["name"] for item in self.survival_items])}

        Remember that this is for a lunar survival scenario where a spacecraft has crash-landed on the moon, 
        approximately 200 miles from the rendezvous point where another spacecraft is waiting.

        The final ranking should represent a holistic evaluation that considers all perspectives.

        Format your response with clear rankings (e.g., "1. Item Name: Justification").
        """

        # Get meta reasoning response
        meta_response = self.meta_agent.chat(meta_prompt)

        # Save meta reasoning to file
        with open("meta_reasoning_response.txt", "w", encoding="utf-8") as f:
            f.write("=== Meta Reasoning Response ===\n\n")
            f.write(meta_response)

        # Parse final ranking
        self.final_ranking = self._parse_ranking(meta_response)

        # Generate comprehensive justifications
        self._generate_comprehensive_justifications()

        logger.info("Meta Reasoning Phase completed.")
        return self.final_ranking
    
    #################################################################################
    

    def _generate_comprehensive_justifications(self):
        """
        Generate comprehensive justifications for each item in the final ranking.
        These justifications integrate technical and practical perspectives and
        provide a complete rationale for the item's position.
        """
        logger.info("Generating comprehensive justifications...")

        # For each item in the final ranking
        for item_ranking in self.final_ranking:
            item_name = item_ranking["name"]
            logger.info(f"Generating comprehensive justification for {item_name}")

            # Get technical and practical perspectives
            tech_perspective = next((item for item in self.technical_ranking if item["name"] == item_name), None)
            prac_perspective = next((item for item in self.practical_ranking if item["name"] == item_name), None)

            tech_just = tech_perspective["justification"] if tech_perspective else "Not provided"
            prac_just = prac_perspective["justification"] if prac_perspective else "Not provided"

            # Create justification prompt
            justification_prompt = f"""
            You need to create a comprehensive justification for {item_name} which is ranked #{item_ranking['rank']} in the final lunar survival items ranking.

            Technical Perspective:
            {tech_just}

            Practical Perspective:
            {prac_just}

            Your current justification:
            {item_ranking['justification']}

            Please integrate these perspectives into a comprehensive, well-structured justification that explains:
            1. The item's function in the lunar environment
            2. Its survival utility and critical timeframes
            3. Its relation to established protocols
            4. Alternative uses and contingency value
            5. Why it deserves its specific ranking position

            This should be thorough yet concise, with clear reasoning.
            """

            # Get comprehensive justification
            comprehensive_justification = self.meta_agent.chat(justification_prompt)

            # Store the comprehensive justification
            self.justifications[item_name] = {
                "rank": item_ranking["rank"],
                "technical_perspective": tech_just,
                "practical_perspective": prac_just,
                "final_justification": comprehensive_justification,
                "confidence": item_ranking["confidence"]
            }

        # Save all justifications to file
        with open("comprehensive_justifications.txt", "w", encoding="utf-8") as f:
            f.write("=== Comprehensive Justifications ===\n\n")
            for item_name, justification in self.justifications.items():
                f.write(f"ITEM: {item_name} (Rank: {justification['rank']})\n\n")
                f.write("FINAL JUSTIFICATION:\n")
                f.write(justification['final_justification'])
                f.write("\n\n" + "-" * 80 + "\n\n")

        return self.justifications


    
    #################################################################################

    def _load_nasa_ranking(self):
        """
        Load NASA's official ranking for the lunar survival challenge.
        This will be used for comparison and scoring.
        """
        # NASA's official ranking (from most to least important)
        nasa_ranking = [
            "Two 100 lb. tanks of oxygen",
            "20 liters of water",
            "Stellar map",
            "Food concentrate",
            "Solar-powered FM receiver-transmitter",
            "50 feet of nylon rope",
            "First aid kit, including injection needle",
            "Parachute silk",
            "Self-inflating life raft",
            "Signal flares",
            "Two .45 caliber pistols",
            "One case of dehydrated milk",
            "Portable heating unit",
            "Magnetic compass",
            "Box of matches"
        ]

        # Convert to dict for easier comparison
        self.nasa_ranking = {item: rank+1 for rank, item in enumerate(nasa_ranking)}

        return self.nasa_ranking
    
        
    #################################################################################

    def _calculate_score(self):
        """
        Calculate score based on comparison with NASA's ranking.
        For each item, add the absolute difference between our rank and NASA's rank.

        Returns:
            Total score and score category
        """
        # Ensure NASA ranking is loaded
        if not hasattr(self, 'nasa_ranking'):
            self._load_nasa_ranking()

        total_score = 0
        item_scores = []

        for item in self.final_ranking:
            nasa_rank = self.nasa_ranking.get(item["name"], 15)  # Default to worst rank if not found
            our_rank = item["rank"]
            difference = abs(nasa_rank - our_rank)
            total_score += difference

            item_scores.append({
                "name": item["name"],
                "our_rank": our_rank,
                "nasa_rank": nasa_rank,
                "difference": difference
            })

        # Determine score category
        if total_score <= 25:
            category = "excellent"
        elif total_score <= 32:
            category = "good"
        elif total_score <= 45:
            category = "average"
        elif total_score <= 55:
            category = "fair"
        elif total_score <= 70:
            category = "poor -- suggests use of Earth-bound logic"
        else:
            category = "very poor – you're one of the casualties of the space program!"

        self.score = {
            "total": total_score,
            "category": category,
            "item_scores": item_scores
        }

        return self.score
    
        
    #################################################################################

    def generate_report(self):
        """
        Generate a comprehensive final report on the lunar survival item rankings.

        Returns:
            A formatted report string
        """
        # Ensure we have final rankings
        if not self.final_ranking:
            raise ValueError("No final ranking available. Run the system first.")

        # Calculate score
        score = self._calculate_score()

        # Sort items by rank
        sorted_items = sorted(self.final_ranking, key=lambda x: x["rank"])

        # Build report header
        report = """
        # Lunar Survival Challenge - Final Report

        ## Executive Summary

        This report presents the final rankings for the lunar survival challenge items based on a collaborative
        multi-agent reasoning system. The system integrated technical perspectives (focusing on scientific principles
        and space procedures) with practical perspectives (focusing on survival needs and contingency planning).

        ### Score Comparison with NASA Ranking

        Total Score: {total_score} - Category: {category}

        ## Methodology

        The evaluation followed a structured collaborative process:

        1. **Knowledge Acquisition Phase**: Science and Resource analysts gathered environmental and survival knowledge
        2. **Technical Reasoning Phase**: Science and Protocol reasoners evaluated items from a technical perspective
        3. **Practical Reasoning Phase**: Resource and Creative reasoners evaluated items from a practical perspective
        4. **Cross-Team Integration**: Team representatives reconciled perspectives and addressed disagreements
        5. **Meta Reasoning**: A holistic analysis integrated all perspectives into final rankings

        ## Final Rankings and NASA Comparison

        """.format(
            total_score=score["total"],
            category=score["category"]
        )

        # Add comparison table
        report += "| Rank | Item | Our Justification | NASA Rank | Difference |\n"
        report += "| ---- | ---- | ---------------- | --------- | ---------- |\n"

        for item in sorted_items:
            nasa_rank = self.nasa_ranking.get(item["name"], "N/A")
            difference = abs(nasa_rank - item["rank"]) if nasa_rank != "N/A" else "N/A"

            # Truncate justification for table
            justification = self.justifications.get(item["name"], {}).get("final_justification", item["justification"])
            short_justification = justification[:100] + "..." if len(justification) > 100 else justification

            report += f"| {item['rank']} | {item['name']} | {short_justification} | {nasa_rank} | {difference} |\n"

        # Add detailed justifications
        report += "\n## Detailed Item Justifications\n\n"

        for item in sorted_items:
            report += f"""
        ### {item['rank']}. {item['name']} (Confidence: {item['confidence']}/10)

        {self.justifications.get(item['name'], {}).get('final_justification', item['justification'])}

        """

        return report
    
        
    #################################################################################

    def save_report_as_pdf(self, filename="lunar_survival_report.pdf"):
        """
        Save the final report to a PDF file.

        Args:
            filename: Name of file to save report to
        """
        # Calculate score if not already done
        if not hasattr(self, 'score'):
            self._calculate_score()

        # Create PDF object
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.add_page()

        # Add title
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, "Lunar Survival Challenge - Final Report", ln=True, align="C")
        pdf.ln(5)

        # Add executive summary
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, "Executive Summary", ln=True)
        pdf.set_font("Arial", "", 11)
        pdf.multi_cell(0, 10, "This report presents the final rankings for the lunar survival challenge items based on a collaborative multi-agent reasoning system. The system integrated technical perspectives with practical perspectives.")
        pdf.ln(5)

        # Add score comparison
        pdf.set_font("Arial", "B", 12)
        pdf.cell(0, 10, f"Score Comparison with NASA Ranking", ln=True)
        pdf.set_font("Arial", "", 11)
        pdf.cell(0, 10, f"Total Score: {self.score['total']} - Category: {self.score['category']}", ln=True)
        pdf.ln(5)

        # Add methodology
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, "Methodology", ln=True)
        pdf.set_font("Arial", "", 11)
        pdf.multi_cell(0, 10, "The evaluation followed a structured collaborative process:")
        pdf.multi_cell(0, 10, "1. Knowledge Acquisition Phase: Science and Resource analysts gathered environmental and survival knowledge")
        pdf.multi_cell(0, 10, "2. Technical Reasoning Phase: Science and Protocol reasoners evaluated items from a technical perspective")
        pdf.multi_cell(0, 10, "3. Practical Reasoning Phase: Resource and Creative reasoners evaluated items from a practical perspective")
        pdf.multi_cell(0, 10, "4. Cross-Team Integration: Team representatives reconciled perspectives and addressed disagreements")
        pdf.multi_cell(0, 10, "5. Meta Reasoning: A holistic analysis integrated all perspectives into final rankings")
        pdf.ln(5)

        # Add final rankings and NASA comparison
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, "Final Rankings and NASA Comparison", ln=True)
        pdf.set_font("Arial", "B", 11)

        # Create comparison table
        col_widths = [15, 70, 15, 25]
        pdf.cell(col_widths[0], 10, "Rank", border=1)
        pdf.cell(col_widths[1], 10, "Item", border=1)
        pdf.cell(col_widths[2], 10, "NASA", border=1)
        pdf.cell(col_widths[3], 10, "Difference", border=1, ln=True)

        # Add table rows
        pdf.set_font("Arial", "", 10)
        for item in sorted(self.final_ranking, key=lambda x: x["rank"]):
            nasa_rank = self.nasa_ranking.get(item["name"], "N/A")
            difference = abs(nasa_rank - item["rank"]) if isinstance(nasa_rank, int) else "N/A"

            pdf.cell(col_widths[0], 10, str(item["rank"]), border=1)
            pdf.cell(col_widths[1], 10, item["name"][:35], border=1)
            pdf.cell(col_widths[2], 10, str(nasa_rank), border=1)
            pdf.cell(col_widths[3], 10, str(difference), border=1, ln=True)

        pdf.ln(10)

        # Add detailed justifications
        pdf.add_page()
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, "Detailed Item Justifications", ln=True)

        for item in sorted(self.final_ranking, key=lambda x: x["rank"]):
            pdf.set_font("Arial", "B", 12)
            pdf.cell(0, 10, f"{item['rank']}. {item['name']} (Confidence: {item['confidence']}/10)", ln=True)

            pdf.set_font("Arial", "", 10)
            justification = self.justifications.get(item["name"], {}).get("final_justification", item["justification"])

            # Split justification into paragraphs for better formatting
            paragraphs = justification.split("\n\n")
            for para in paragraphs:
                if para.strip():
                    pdf.multi_cell(0, 5, para)
                    pdf.ln(2)

            pdf.ln(5)

        # Save PDF
        pdf.output(filename)
        logger.info(f"Report saved as PDF to {filename}")

        # Also save detailed agent discussions to a separate PDF for review
        self._save_agent_discussions_pdf()

        return filename
    
        
    #################################################################################

    def _save_agent_discussions_pdf(self, filename="lunar_survival_agent_discussions.pdf"):
        """Save all agent discussions to a separate PDF for detailed review."""
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)

        # Add knowledge acquisition discussion
        pdf.add_page()
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, "Knowledge Acquisition Team Discussion", ln=True)

        if hasattr(self.knowledge_team, "team_conversation"):
            for entry in self.knowledge_team.team_conversation:
                pdf.set_font("Arial", "B", 12)
                pdf.cell(0, 10, f"Round: {entry['round']} - Agent: {entry['agent']}", ln=True)

                pdf.set_font("Arial", "", 10)
                # Truncate very long messages to avoid PDF issues
                message = entry['message'][:5000] + "..." if len(entry['message']) > 5000 else entry['message']
                pdf.multi_cell(0, 5, message)
                pdf.ln(5)

        # Add technical reasoning discussion
        pdf.add_page()
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, "Technical Reasoning Team Discussion", ln=True)

        if hasattr(self.technical_team, "team_conversation"):
            for entry in self.technical_team.team_conversation:
                pdf.set_font("Arial", "B", 12)
                pdf.cell(0, 10, f"Round: {entry['round']} - Agent: {entry['agent']}", ln=True)

                pdf.set_font("Arial", "", 10)
                message = entry['message'][:5000] + "..." if len(entry['message']) > 5000 else entry['message']
                pdf.multi_cell(0, 5, message)
                pdf.ln(5)

        # Add practical reasoning discussion
        pdf.add_page()
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, "Practical Reasoning Team Discussion", ln=True)

        if hasattr(self.practical_team, "team_conversation"):
            for entry in self.practical_team.team_conversation:
                pdf.set_font("Arial", "B", 12)
                pdf.cell(0, 10, f"Round: {entry['round']} - Agent: {entry['agent']}", ln=True)

                pdf.set_font("Arial", "", 10)
                message = entry['message'][:5000] + "..." if len(entry['message']) > 5000 else entry['message']
                pdf.multi_cell(0, 5, message)
                pdf.ln(5)

        # Add integration discussion
        pdf.add_page()
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, "Cross-Team Integration Discussion", ln=True)

        if hasattr(self, "integration_discussion"):
            for entry in self.integration_discussion:
                pdf.set_font("Arial", "B", 12)
                pdf.cell(0, 10, f"Round: {entry['round']} - Agent: {entry['agent']}", ln=True)

                pdf.set_font("Arial", "", 10)
                message = entry['message'][:5000] + "..." if len(entry['message']) > 5000 else entry['message']
                pdf.multi_cell(0, 5, message)
                pdf.ln(5)

        # Save PDF
        pdf.output(filename)
        logger.info(f"Agent discussions saved as PDF to {filename}")
        
            
    #################################################################################

    def run(self) -> Dict[str, Any]:
        """
        Run the complete lunar survival challenge analysis.

        Returns:
            Dict containing final rankings and justifications
        """
        # Step 1: Knowledge Acquisition Phase
        self._knowledge_acquisition_phase()

        # Step 2: Technical Team Reasoning
        self._technical_reasoning_phase()

        # Step 3: Practical Team Reasoning
        self._practical_reasoning_phase()

        # Step 4: Cross-Team Integration
        self._cross_team_integration()

        # Step 5: Meta Reasoning and Final Ranking
        self._meta_reasoning_phase()

        # Prepare and return results
        results = {
            "final_ranking": self.final_ranking,
            "justifications": self.justifications,
            "technical_ranking": self.technical_ranking,
            "practical_ranking": self.practical_ranking
        }

        return results

## Main Execution function

In [20]:
#!/usr/bin/env python3
"""
Lunar Survival Challenge - Multi-Agent Collaborative System
"""

import os
import time
import json
import logging
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.FileHandler("lunar_survival_system.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)



In [21]:
"""Main execution function"""

# Load environment variables
load_dotenv()

# Check for API key
if not os.environ.get('AZURE_OPENAI_VARE_KEY') or not os.environ.get('AZURE_ENDPOINT'):
    logger.error("Error: AZURE_OPENAI_VARE_KEY or AZURE_ENDPOINT environment variable not set.")
    logger.error("Please set your Azure OpenAI API key in a .env file or environment variable.")

logger.info("=== Lunar Survival Challenge - Multi-Agent System ===")
logger.info("Initializing system...")

# Create the lunar survival system
lunar_system = LunarSurvivalSystem()

# Track timing
start_time = time.time()

try:
    # Run the complete analysis
    logger.info("Running complete lunar survival analysis...")
    results = lunar_system.run()

    # Generate and save the report as PDF
    logger.info("Generating final report...")
    #lunar_system.save_report_as_pdf()

    # Save raw results as JSON
    with open("lunar_survival_results.json", "w", encoding="utf-8") as f:
        # Convert complex objects to serializable format
        serializable_results = {
            "final_ranking": results["final_ranking"],
            "technical_ranking": results["technical_ranking"],
            "practical_ranking": results["practical_ranking"],
        }
        json.dump(serializable_results, f, indent=2)

    # Print final rankings and NASA comparison
    logger.info("\n=== Final Rankings vs NASA ===")
    nasa_ranking = lunar_system._load_nasa_ranking()
    score = lunar_system._calculate_score()

    logger.info(f"TOTAL SCORE: {score['total']} - Category: {score['category']}")
    logger.info("-" * 50)

    for item in sorted(results["final_ranking"], key=lambda x: x["rank"]):
        nasa_rank = nasa_ranking.get(item["name"], "N/A")
        diff = abs(nasa_rank - item["rank"]) if isinstance(nasa_rank, int) else "N/A"
        logger.info(f"{item['rank']}. {item['name']} (NASA: {nasa_rank}, Diff: {diff})")

    # Print timing information
    end_time = time.time()
    duration = end_time - start_time
    hours, remainder = divmod(duration, 3600)
    minutes, seconds = divmod(remainder, 60)

    logger.info(f"\nAnalysis completed in {int(hours)}h {int(minutes)}m {int(seconds)}s.")
    logger.info(f"Final report saved to lunar_survival_report.pdf")
    logger.info(f"Raw results saved to lunar_survival_results.json")
    logger.info(f"Agent discussions saved to lunar_survival_agent_discussions.pdf")
    logger.info(f"Detailed logs saved to various .txt files for review")

except Exception as e:
    logger.error(f"Error during execution: {str(e)}", exc_info=True)
    raise

2025-03-12 03:36:41,457 [INFO] === Lunar Survival Challenge - Multi-Agent System ===
2025-03-12 03:36:41,458 [INFO] Initializing system...
2025-03-12 03:36:41,458 [INFO] Initializing Lunar Survival System
2025-03-12 03:36:41,485 [INFO] Initialized Web Researcher agent
2025-03-12 03:36:41,486 [INFO] Initializing Knowledge Acquisition team with goal: gather and analyze information about the lunar environment and human survival needs
2025-03-12 03:36:41,512 [INFO] Initialized Science Analyst agent
2025-03-12 03:36:41,538 [INFO] Initialized Resource Analyst agent
2025-03-12 03:36:41,539 [INFO] Initializing Technical Reasoning team with goal: evaluate items based on scientific principles and established space procedures
2025-03-12 03:36:41,556 [INFO] Initialized Science Reasoner agent
2025-03-12 03:36:41,574 [INFO] Initialized Protocol Reasoner agent
2025-03-12 03:36:41,574 [INFO] Initializing Practical Reasoning team with goal: evaluate items based on survival needs and contingency plannin

2025-03-12 03:38:32,358 [INFO] [Science Analyst] Received message: As a Science Analyst, please provide your initial thoughts on: 
        You need to collaborate to c...
2025-03-12 03:39:04,469 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-12 03:39:04,474 [INFO] [Science Analyst] Responded: ### Integrated Knowledge Framework for Lunar Survival

This framework connects the **lunar environme...
2025-03-12 03:39:04,474 [INFO] [Resource Analyst] Received message: As a Resource Analyst, please provide your initial thoughts on: 
        You need to collaborate to ...
2025-03-12 03:39:29,513 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-12 03:39:29,518 [INFO] [Resource Analyst] Responded: ### Integrated Knowl

2025-03-12 03:42:37,850 [INFO] [Science Analyst] Received message: As a Science Analyst, please provide your initial thoughts on: 
        Based on the lunar environme...
2025-03-12 03:42:38,729 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 429 Too Many Requests"
2025-03-12 03:42:38,731 [INFO] Retrying request to /chat/completions in 32.000000 seconds
2025-03-12 03:43:29,018 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-12 03:43:29,020 [INFO] [Science Analyst] Responded: ### Analysis of Survival Items for the Lunar Environment  

Below is a concise evaluation of each su...
2025-03-12 03:43:29,021 [INFO] [Resource Analyst] Received message: As a Resource Analyst, please provide your initial thoughts on: 
        Based o

2025-03-12 03:46:39,285 [INFO] [Science Reasoner] Received message: As a Science Reasoner, please provide your initial thoughts on: 
        You are tasked with evaluat...
2025-03-12 03:46:50,862 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-12 03:46:50,864 [INFO] [Science Reasoner] Responded: ### Integrated Technical Ranking for Lunar Survival Items  

This ranking evaluates the functionalit...
2025-03-12 03:46:50,866 [INFO] [Protocol Reasoner] Received message: As a Protocol Reasoner, please provide your initial thoughts on: 
        You are tasked with evalua...
2025-03-12 03:47:07,793 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-12 03:47:07,796 [INFO] [Protocol Reasoner] Responded: ### Integrated T

2025-03-12 03:49:59,421 [INFO] [Resource Reasoner] Received message: As a Resource Reasoner, please provide your initial thoughts on: 
        You are tasked with evalua...
2025-03-12 03:50:19,541 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-12 03:50:19,546 [INFO] [Resource Reasoner] Responded: ### Integrated Practical Ranking of Lunar Survival Items  

Below is the ranking of the items based ...
2025-03-12 03:50:19,547 [INFO] [Creative Reasoner] Received message: As a Creative Reasoner, please provide your initial thoughts on: 
        You are tasked with evalua...
2025-03-12 03:50:29,207 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-12 03:50:29,210 [INFO] [Creative Reasoner] Responded: ### Integrated

2025-03-12 03:54:06,072 [INFO] [Technical Representative] Received message: As a Technical Representative, please provide your initial thoughts on: 
        Both the Technical ...
2025-03-12 03:54:46,022 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-12 03:54:46,026 [INFO] [Technical Representative] Responded: ### Initial Thoughts as a Technical Representative:

The discrepancies between the **Technical Team*...
2025-03-12 03:54:46,027 [INFO] [Practical Representative] Received message: As a Practical Representative, please provide your initial thoughts on: 
        Both the Technical ...
2025-03-12 03:55:18,937 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-12 03:55:18,941 [INFO] [Practical Representativ

2025-03-12 03:59:18,378 [INFO] [Meta Reasoner] Received message: 
            You need to create a comprehensive justification for Food concentrate which is ranked #...
2025-03-12 03:59:35,793 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-12 03:59:35,797 [INFO] [Meta Reasoner] Responded: ### Comprehensive Justification for Ranking Food Concentrate as #2

---

#### **1. The Item's Functi...
2025-03-12 03:59:35,798 [INFO] Generating comprehensive justification for 50 feet of nylon rope
2025-03-12 03:59:35,799 [INFO] [Meta Reasoner] Received message: 
            You need to create a comprehensive justification for 50 feet of nylon rope which is ran...
2025-03-12 03:59:44,160 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 

2025-03-12 04:02:08,141 [INFO] [Meta Reasoner] Received message: 
            You need to create a comprehensive justification for Solar-powered FM receiver-transmit...
2025-03-12 04:02:08,309 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 429 Too Many Requests"
2025-03-12 04:02:08,311 [INFO] Retrying request to /chat/completions in 3.000000 seconds
2025-03-12 04:02:21,382 [INFO] HTTP Request: POST https://vare-labs-azure-openai-resource.openai.azure.com/openai/deployments/VARELab-GPT4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-03-12 04:02:21,386 [INFO] [Meta Reasoner] Responded: ### Comprehensive Justification for Ranking Solar-Powered FM Receiver-Transmitter as #15

---

#### ...
2025-03-12 04:02:21,390 [INFO] Meta Reasoning Phase completed.
2025-03-12 04:02:21,391 [INFO] Generating final report...
2025-03-12 04:02:21,397 [INFO] 
